# Model Deployment with Hugging Face 🚀

Deployment brings your models from development to production, making them accessible to real users and applications.

## What is Model Deployment?

**Model Deployment** makes models production-ready:
- **Serving**: Host models via APIs
- **Scaling**: Handle multiple concurrent requests
- **Monitoring**: Track performance and usage
- **Examples**: REST APIs, web apps, mobile integration

## Learning Objectives

By the end of this notebook, you'll know how to:
1. Deploy models using Hugging Face Inference API
2. Create REST APIs with FastAPI and Flask
3. Containerize models with Docker
4. Handle scaling and load balancing
5. Monitor model performance in production
6. Implement A/B testing for model updates

Let's deploy! 🌐

In [ ]:
# Import essential libraries
from transformers import pipeline
import requests
import json
print('Deployment notebook ready!')

## 1. Hugging Face Inference API

The simplest deployment method - use Hugging Face's hosted infrastructure.

In [ ]:
# Using the Inference API for sentiment analysis
API_URL = "https://api-inference.huggingface.co/models/cardiffnlp/twitter-roberta-base-sentiment-latest"
headers = {"Authorization": "Bearer YOUR_HF_TOKEN_HERE"}  # Replace with your token

def query_inference_api(payload):
    """Send request to Hugging Face Inference API"""
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Example usage
test_texts = [
    "I love using Hugging Face models!",
    "This deployment tutorial is confusing",
    "The weather is okay today"
]

print("🔥 Hugging Face Inference API Results:")
for text in test_texts:
    data = {"inputs": text}
    result = query_inference_api(data)
    print(f"\nText: '{text}'")
    print(f"Result: {result}")
    
print("\n✅ Inference API deployment complete!")
print("💡 Pros: Zero setup, managed scaling, built-in caching")
print("⚠️ Cons: Rate limits, cold starts, less control")

## 2. FastAPI REST API

Create a custom API service for full control over your deployment.

In [ ]:
# Create FastAPI application code
fastapi_code = '''
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel
from transformers import pipeline
import time
import logging
from typing import List

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(
    title="HF Model API", 
    description="Production-ready Hugging Face model deployment",
    version="1.0.0"
)

# Load model once at startup
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    return_all_scores=True
)

# Request/Response models
class PredictionRequest(BaseModel):
    text: str
    
class BatchRequest(BaseModel):
    texts: List[str]

class PredictionResponse(BaseModel):
    text: str
    predictions: List[dict]
    processing_time: float

# Metrics tracking
request_count = 0

def log_request():
    global request_count
    request_count += 1
    logger.info(f"Processing request #{request_count}")

@app.get("/")
async def root():
    return {
        "message": "HF Model API is running!",
        "model": "twitter-roberta-base-sentiment-latest",
        "total_requests": request_count
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest, background_tasks: BackgroundTasks):
    background_tasks.add_task(log_request)
    
    start_time = time.time()
    
    try:
        result = classifier(request.text)
        processing_time = time.time() - start_time
        
        return PredictionResponse(
            text=request.text,
            predictions=result[0],
            processing_time=processing_time
        )
    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/batch_predict")
async def batch_predict(request: BatchRequest, background_tasks: BackgroundTasks):
    background_tasks.add_task(log_request)
    
    start_time = time.time()
    results = []
    
    for text in request.texts:
        try:
            result = classifier(text)
            results.append({
                "text": text,
                "predictions": result[0]
            })
        except Exception as e:
            results.append({
                "text": text,
                "error": str(e)
            })
    
    processing_time = time.time() - start_time
    
    return {
        "results": results,
        "processing_time": processing_time,
        "total_texts": len(request.texts)
    }

@app.get("/health")
async def health_check():
    return {"status": "healthy", "timestamp": time.time()}

@app.get("/metrics")
async def get_metrics():
    return {
        "total_requests": request_count,
        "model_loaded": classifier is not None,
        "uptime": time.time()
    }

# To run: uvicorn app:app --host 0.0.0.0 --port 8000 --reload
'''

# Save the FastAPI code
with open('fastapi_deployment.py', 'w') as f:
    f.write(fastapi_code)

print("📝 FastAPI deployment code saved to 'fastapi_deployment.py'")
print("\n🚀 To run the API:")
print("1. pip install fastapi uvicorn")
print("2. uvicorn fastapi_deployment:app --host 0.0.0.0 --port 8000 --reload")
print("3. Visit http://localhost:8000/docs for interactive API documentation")
print("\n✨ Features:")
print("- Single and batch predictions")
print("- Built-in metrics tracking")
print("- Health checks")
print("- Request logging")
print("- Auto-generated documentation")

## 3. Docker Containerization

Package your model in a Docker container for consistent, portable deployment.

In [ ]:
# Create Dockerfile for model deployment
dockerfile_content = '''
# Multi-stage build for smaller final image
FROM python:3.9-slim as builder

WORKDIR /app

# Install build dependencies
RUN apt-get update && apt-get install -y \\
    gcc \\
    g++ \\
    && rm -rf /var/lib/apt/lists/*

# Install Python dependencies
COPY requirements.txt .
RUN pip install --user --no-cache-dir -r requirements.txt

# Production stage
FROM python:3.9-slim

WORKDIR /app

# Copy installed packages from builder
COPY --from=builder /root/.local /root/.local

# Copy application code
COPY fastapi_deployment.py .

# Create non-root user for security
RUN useradd --create-home --shell /bin/bash app \\
    && chown -R app:app /app
USER app

# Download model at build time (optional - saves startup time)
RUN python -c "from transformers import pipeline; pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment-latest')"

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=30s --retries=3 \\
    CMD curl -f http://localhost:8000/health || exit 1

EXPOSE 8000

# Use exec form for better signal handling
CMD ["python", "-m", "uvicorn", "fastapi_deployment:app", "--host", "0.0.0.0", "--port", "8000"]
'''

# Create requirements.txt for Docker
requirements_content = '''
fastapi==0.104.1
uvicorn[standard]==0.24.0
transformers==4.35.2
torch==2.1.0
pydantic==2.5.0
requests==2.31.0
'''

# Create docker-compose for easier orchestration
docker_compose_content = '''
version: '3.8'

services:
  huggingface-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - PYTHONUNBUFFERED=1
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
    
  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
    depends_on:
      - huggingface-api
    restart: unless-stopped
'''

# Create nginx config for load balancing
nginx_config = '''
events {
    worker_connections 1024;
}

http {
    upstream app {
        server huggingface-api:8000;
    }

    server {
        listen 80;
        
        location / {
            proxy_pass http://app;
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;
            
            # Timeouts
            proxy_connect_timeout 60s;
            proxy_send_timeout 60s;
            proxy_read_timeout 60s;
        }
        
        location /health {
            proxy_pass http://app/health;
            access_log off;
        }
    }
}
'''

# Save all Docker-related files
files_to_create = {
    'Dockerfile': dockerfile_content,
    'requirements.txt': requirements_content,
    'docker-compose.yml': docker_compose_content,
    'nginx.conf': nginx_config
}

for filename, content in files_to_create.items():
    with open(filename, 'w') as f:
        f.write(content)

print("🐳 Docker deployment files created successfully!")
print("\n📦 Files created:")
for filename in files_to_create.keys():
    print(f"  - {filename}")

print("\n🚀 To deploy:")
print("1. docker build -t huggingface-api .")
print("2. docker run -p 8000:8000 huggingface-api")
print("\n🔄 Or use docker-compose:")
print("1. docker-compose up --build")
print("2. Access API at http://localhost:80")

print("\n✨ Features:")
print("- Multi-stage build for smaller image")
print("- Non-root user for security")
print("- Health checks")
print("- Nginx reverse proxy")
print("- Production-ready configuration")

## 4. Gradio Web Interface

Create an interactive web demo for your model.

In [ ]:
# Create Gradio web interface
gradio_code = '''
import gradio as gr
from transformers import pipeline
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load model
print("Loading model...")
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    return_all_scores=True
)
print("Model loaded successfully!")

# Analytics tracking
usage_stats = {
    "total_predictions": 0,
    "start_time": time.time()
}

def analyze_sentiment(text, return_confidence=True):
    """Analyze sentiment with detailed results"""
    if not text.strip():
        return "⚠️ Please enter some text to analyze.", None
    
    start_time = time.time()
    
    try:
        # Get predictions
        results = classifier(text)
        processing_time = time.time() - start_time
        
        # Update usage stats
        usage_stats["total_predictions"] += 1
        
        # Format results
        formatted_results = []
        confidence_data = []
        
        for result in results[0]:
            label = result['label'].replace('LABEL_', '')
            if label == '0':
                label = '😢 Negative'
            elif label == '1':
                label = '😐 Neutral'
            elif label == '2':
                label = '😊 Positive'
            
            score = result['score']
            formatted_results.append(f"{label}: {score:.3f} ({score*100:.1f}%)")
            confidence_data.append((label, score))
        
        # Create response
        response = "\n".join(formatted_results)
        response += f"\n\n⚡ Processing time: {processing_time:.3f}s"
        response += f"\n📊 Total predictions: {usage_stats['total_predictions']}"
        
        # Log the prediction
        logger.info(f"Prediction #{usage_stats['total_predictions']}: {processing_time:.3f}s")
        
        if return_confidence:
            return response, confidence_data
        else:
            return response, None
            
    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        logger.error(error_msg)
        return error_msg, None

def get_usage_stats():
    """Get current usage statistics"""
    uptime = time.time() - usage_stats["start_time"]
    hours = int(uptime // 3600)
    minutes = int((uptime % 3600) // 60)
    
    return f"""📈 Usage Statistics:
• Total Predictions: {usage_stats['total_predictions']}
• Uptime: {hours}h {minutes}m
• Model: twitter-roberta-base-sentiment-latest
• Status: ✅ Running"""

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="HF Sentiment Analysis") as demo:
    gr.Markdown("""
    # 🤗 Hugging Face Sentiment Analysis
    
    Analyze the sentiment of any text using a RoBERTa model fine-tuned on Twitter data.
    This model can detect **positive**, **negative**, and **neutral** sentiments.
    """)
    
    with gr.Tab("Single Prediction"):
        with gr.Row():
            with gr.Column():
                text_input = gr.Textbox(
                    lines=4,
                    placeholder="Enter text to analyze sentiment...",
                    label="Input Text"
                )
                confidence_checkbox = gr.Checkbox(
                    label="Show confidence chart", 
                    value=True
                )
                analyze_btn = gr.Button("🔍 Analyze Sentiment", variant="primary")
                
            with gr.Column():
                result_output = gr.Textbox(
                    label="Analysis Results",
                    lines=6,
                    interactive=False
                )
                confidence_chart = gr.BarPlot(
                    label="Confidence Scores",
                    x="label",
                    y="score",
                    title="Sentiment Confidence",
                    y_lim=[0, 1]
                )
    
    with gr.Tab("Batch Analysis"):
        batch_input = gr.Textbox(
            lines=8,
            placeholder="Enter multiple texts, one per line...",
            label="Multiple Texts"
        )
        batch_btn = gr.Button("🔍 Analyze All", variant="primary")
        batch_output = gr.Textbox(
            label="Batch Results",
            lines=10,
            interactive=False
        )
    
    with gr.Tab("Statistics"):
        stats_output = gr.Textbox(
            label="Usage Statistics",
            lines=6,
            interactive=False
        )
        refresh_stats_btn = gr.Button("🔄 Refresh Stats")
    
    # Event handlers
    analyze_btn.click(
        fn=analyze_sentiment,
        inputs=[text_input, confidence_checkbox],
        outputs=[result_output, confidence_chart]
    )
    
    def batch_analyze(batch_text):
        if not batch_text.strip():
            return "⚠️ Please enter texts to analyze."
        
        texts = [line.strip() for line in batch_text.split('\n') if line.strip()]
        results = []
        
        for i, text in enumerate(texts, 1):
            result, _ = analyze_sentiment(text, return_confidence=False)
            results.append(f"**Text {i}:** {text[:50]}{'...' if len(text) > 50 else ''}")
            results.append(result)
            results.append("---")
        
        return "\n".join(results)
    
    batch_btn.click(
        fn=batch_analyze,
        inputs=batch_input,
        outputs=batch_output
    )
    
    refresh_stats_btn.click(
        fn=get_usage_stats,
        outputs=stats_output
    )
    
    # Examples
    gr.Examples(
        examples=[
            ["I absolutely love this new feature! It's amazing! 🎉"],
            ["This is terrible and completely disappointing."],
            ["The weather is okay today, nothing special."],
            ["Just deployed my first ML model using Hugging Face! 🚀"],
            ["The meeting was cancelled again. So frustrating! 😤"]
        ],
        inputs=text_input,
        label="Example Texts"
    )

# Launch the interface
if __name__ == "__main__":
    demo.launch(
        share=False,  # Set to True for public links
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )
'''

# Save Gradio code
with open('gradio_demo.py', 'w') as f:
    f.write(gradio_code)

print("🎨 Gradio web interface created successfully!")
print("\n📁 File saved: gradio_demo.py")
print("\n🚀 To run:")
print("1. pip install gradio")
print("2. python gradio_demo.py")
print("3. Open http://localhost:7860 in your browser")

print("\n✨ Features:")
print("- Interactive web interface")
print("- Single and batch predictions")
print("- Confidence visualization")
print("- Usage statistics")
print("- Example texts")
print("- Mobile-friendly design")
print("- Real-time processing")

## 5. Load Testing & Monitoring

Test your deployment's performance and implement monitoring.

In [ ]:
# Create load testing script
load_test_code = '''
import requests
import time
import concurrent.futures
import statistics
import json
from typing import List, Dict

class LoadTester:
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url
        self.results = []
    
    def send_request(self, text: str, request_id: int) -> Dict:
        """Send a single prediction request"""
        start_time = time.time()
        
        try:
            response = requests.post(
                f"{self.base_url}/predict",
                json={"text": text},
                timeout=30
            )
            
            end_time = time.time()
            
            return {
                "request_id": request_id,
                "status_code": response.status_code,
                "response_time": end_time - start_time,
                "success": response.status_code == 200,
                "timestamp": start_time
            }
        
        except Exception as e:
            end_time = time.time()
            return {
                "request_id": request_id,
                "status_code": 0,
                "response_time": end_time - start_time,
                "success": False,
                "error": str(e),
                "timestamp": start_time
            }
    
    def run_load_test(self, 
                     test_texts: List[str],
                     num_requests: int = 100,
                     concurrent_users: int = 10) -> Dict:
        """Run load test with multiple concurrent requests"""
        
        print(f"🔥 Starting load test...")
        print(f"📊 Requests: {num_requests}, Concurrent users: {concurrent_users}")
        
        # Prepare requests
        requests_to_send = []
        for i in range(num_requests):
            text = test_texts[i % len(test_texts)]
            requests_to_send.append((text, i))
        
        # Execute concurrent requests
        results = []
        start_time = time.time()
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_users) as executor:
            future_to_request = {
                executor.submit(self.send_request, text, req_id): (text, req_id)
                for text, req_id in requests_to_send
            }
            
            for future in concurrent.futures.as_completed(future_to_request):
                result = future.result()
                results.append(result)
        
        end_time = time.time()
        total_time = end_time - start_time
        
        # Calculate statistics
        successful_requests = [r for r in results if r['success']]
        failed_requests = [r for r in results if not r['success']]
        
        response_times = [r['response_time'] for r in successful_requests]
        
        stats = {
            "total_requests": num_requests,
            "successful_requests": len(successful_requests),
            "failed_requests": len(failed_requests),
            "success_rate": len(successful_requests) / num_requests * 100,
            "total_time": total_time,
            "requests_per_second": num_requests / total_time,
            "avg_response_time": statistics.mean(response_times) if response_times else 0,
            "median_response_time": statistics.median(response_times) if response_times else 0,
            "min_response_time": min(response_times) if response_times else 0,
            "max_response_time": max(response_times) if response_times else 0,
            "p95_response_time": statistics.quantiles(response_times, n=20)[18] if len(response_times) > 20 else 0,
            "p99_response_time": statistics.quantiles(response_times, n=100)[98] if len(response_times) > 100 else 0
        }
        
        return stats
    
    def print_results(self, stats: Dict):
        """Print formatted test results"""
        print("\n" + "="*60)
        print("🎯 LOAD TEST RESULTS")
        print("="*60)
        
        print(f"\n📊 Overview:")
        print(f"  Total Requests: {stats['total_requests']}")
        print(f"  Successful: {stats['successful_requests']} ({stats['success_rate']:.1f}%)")
        print(f"  Failed: {stats['failed_requests']}")
        print(f"  Total Time: {stats['total_time']:.2f}s")
        
        print(f"\n⚡ Performance:")
        print(f"  Requests/sec: {stats['requests_per_second']:.2f}")
        print(f"  Avg Response Time: {stats['avg_response_time']:.3f}s")
        print(f"  Median Response Time: {stats['median_response_time']:.3f}s")
        
        print(f"\n📈 Response Time Distribution:")
        print(f"  Min: {stats['min_response_time']:.3f}s")
        print(f"  P95: {stats['p95_response_time']:.3f}s")
        print(f"  P99: {stats['p99_response_time']:.3f}s")
        print(f"  Max: {stats['max_response_time']:.3f}s")
        
        # Performance recommendations
        print(f"\n💡 Recommendations:")
        if stats['success_rate'] < 95:
            print(f"  ⚠️ Success rate is low ({stats['success_rate']:.1f}%). Check for errors.")
        if stats['avg_response_time'] > 1.0:
            print(f"  ⚠️ High average response time ({stats['avg_response_time']:.3f}s). Consider optimization.")
        if stats['requests_per_second'] < 10:
            print(f"  ⚠️ Low throughput ({stats['requests_per_second']:.2f} req/s). Consider scaling.")
        if stats['success_rate'] >= 95 and stats['avg_response_time'] <= 1.0:
            print(f"  ✅ Performance looks good!")

# Example usage
if __name__ == "__main__":
    # Test texts for load testing
    test_texts = [
        "I love this product! It's amazing!",
        "This is terrible and disappointing.",
        "The weather is okay today.",
        "Just deployed my first model!",
        "The meeting was cancelled again.",
        "Great work on this project!",
        "I'm not sure about this decision.",
        "The performance is outstanding!",
        "This could be better.",
        "Neutral feedback on the implementation."
    ]
    
    # Create load tester
    tester = LoadTester()
    
    # Test different load scenarios
    scenarios = [
        {"requests": 50, "concurrent": 5, "name": "Light Load"},
        {"requests": 100, "concurrent": 10, "name": "Medium Load"},
        {"requests": 200, "concurrent": 20, "name": "Heavy Load"}
    ]
    
    for scenario in scenarios:
        print(f"\n🧪 Running {scenario['name']} Test...")
        stats = tester.run_load_test(
            test_texts,
            num_requests=scenario['requests'],
            concurrent_users=scenario['concurrent']
        )
        tester.print_results(stats)
        
        # Brief pause between tests
        time.sleep(2)
    
    print("\n🎉 Load testing completed!")
'''

# Save load testing code
with open('load_test.py', 'w') as f:
    f.write(load_test_code)

print("🔧 Load testing script created successfully!")
print("\n📁 File saved: load_test.py")
print("\n🧪 To run load tests:")
print("1. Start your FastAPI server: uvicorn fastapi_deployment:app")
print("2. Run tests: python load_test.py")

print("\n📊 Test scenarios:")
print("- Light Load: 50 requests, 5 concurrent users")
print("- Medium Load: 100 requests, 10 concurrent users") 
print("- Heavy Load: 200 requests, 20 concurrent users")

print("\n✨ Metrics tracked:")
print("- Success rate and error rate")
print("- Response time statistics (avg, median, p95, p99)")
print("- Throughput (requests per second)")
print("- Performance recommendations")